In [ ]:
import meshio
import pyvista as pv
import numpy as np
import os
import traceback # エラー発生時の詳細情報表示用

# PyVistaのJupyter Notebook用バックエンドを設定します。
# 'panel' を使用すると、セル内にインタラクティブなプロットが表示されます。
# もし 'panel' で問題が発生する場合や、他のバックエンドを試したい場合は、
# 'static' (静止画像)、'ipygany'、またはこの行をコメントアウトして
# pv.set_jupyter_backend(None) # (デフォルトの別ウィンドウ表示) などを試してください。
try:
    pv.global_theme.jupyter_backend = 'None'
except Exception as e:
    print(f"PyVista Jupyterバックエンド 'panel' の設定に失敗しました: {e}")
    print("デフォルトのバックエンドまたは 'static' を試みます。")
    try:
        pv.global_theme.jupyter_backend = 'static'
    except:
        print("PyVistaのJupyterバックエンド設定に失敗しました。プロットは別ウィンドウで表示される可能性があります。")


# プロットのテーマを設定（任意、見た目を調整します）
pv.set_plot_theme("document")

In [ ]:
# --- 設定 ---
FILE_NAME = "cylinder_3d.msh" # ローカルファイル名を指定

# --- ローカルファイルの存在確認 ---
if not os.path.exists(FILE_NAME):
    print(f"エラー: メッシュファイル '{FILE_NAME}' が見つかりません。")
    print(f"このNotebookと同じディレクトリに '{FILE_NAME}' を配置してください。")
    # Jupyter Notebookでは処理をここで止めるために例外を発生させます
    raise FileNotFoundError(f"Mesh file '{FILE_NAME}' not found. Please place it in the same directory as this notebook.")
else:
    print(f"ローカルメッシュファイルを使用します: '{FILE_NAME}'")

In [ ]:
print(f"\n--- ステップ 1: `meshio` でメッシュを読み込み中 '{FILE_NAME}' ---")
mesh_from_meshio = None # エラー発生時に備えて初期化
try:
    mesh_from_meshio = meshio.read(FILE_NAME)
    print("meshio によるメッシュの読み込み成功。")
    print("\nmeshio メッシュオブジェクト情報:")
    print(mesh_from_meshio) # オブジェクト自体を出力
    print(f"  点の数: {len(mesh_from_meshio.points)}")
    print("  セルタイプと数:")
    for cell_block in mesh_from_meshio.cells:
        print(f"    - タイプ: '{cell_block.type}', 数: {len(cell_block.data)}")
    
    if mesh_from_meshio.point_data:
        print(f"  点データキー: {list(mesh_from_meshio.point_data.keys())}")
    
    # --- ↓↓↓ ここからが修正箇所 ↓↓↓ ---
    if mesh_from_meshio.cell_data:
        print("  セルデータ (セルタイプ毎):")
        for cell_type, data_for_cell_type in mesh_from_meshio.cell_data.items():
            print(f"    - セルタイプ '{cell_type}':")
            if isinstance(data_for_cell_type, dict):
                # 期待される形式: {'フィールド名1': データ配列1, 'フィールド名2': データ配列2, ...}
                print(f"        データフィールドキー: {list(data_for_cell_type.keys())}")
                for field_name, data_array in data_for_cell_type.items():
                    if hasattr(data_array, 'shape') and hasattr(data_array, 'dtype'):
                        print(f"          - フィールド '{field_name}': shape {data_array.shape}, dtype {data_array.dtype}")
                    else:
                        print(f"          - フィールド '{field_name}': {data_array} (型: {type(data_array).__name__})")
            elif isinstance(data_for_cell_type, list):
                # エラーが示唆する形式: [データ配列1, データ配列2, ...] または [フィールド名1, フィールド名2, ...]
                print(f"        データ (リスト形式、要素数: {len(data_for_cell_type)}):")
                for i, item in enumerate(data_for_cell_type):
                    if hasattr(item, 'shape') and hasattr(item, 'dtype'): # NumPy配列と仮定
                        print(f"          - 要素 {i}: shape {item.shape}, dtype {item.dtype}")
                    else:
                        print(f"          - 要素 {i}: {str(item)[:80]}{'...' if len(str(item)) > 80 else ''} (型: {type(item).__name__})") # 長すぎる場合は省略
            else:
                # その他の予期しない形式
                print(f"        データ (不明な形式): {str(data_for_cell_type)[:80]}{'...' if len(str(data_for_cell_type)) > 80 else ''} (型: {type(data_for_cell_type).__name__})")
    # --- ↑↑↑ ここまでが修正箇所 ↑↑↑ ---

except Exception as e:
    print(f"meshio でのメッシュ読み込み中または情報表示中にエラー発生: {e}")
    traceback.print_exc()

In [ ]:
print("\n--- ステップ 3 (オプション): `meshio` オブジェクトから `PyVista UnstructuredGrid` への手動変換 ---")

# meshioのセルタイプ文字列をPyVistaのVTKセルタイプEnumにマッピング
meshio_str_to_pv_celltype = {
    "vertex": pv.CellType.VERTEX,
    "line": pv.CellType.LINE,
    "triangle": pv.CellType.TRIANGLE,
    "quad": pv.CellType.QUAD,
    "tetra": pv.CellType.TETRA,
    "pyramid": pv.CellType.PYRAMID,
    "wedge": pv.CellType.WEDGE,
    "hexahedron": pv.CellType.HEXAHEDRON,
    # --- ↓↓↓ 'tetra20' のためのマッピングを追加 ↓↓↓ ---
    "tetra10": pv.CellType.QUADRATIC_TETRA, # 10節点二次四面体 (参考)
    "tetra20": pv.CellType.LAGRANGE_TETRAHEDRON, # 20節点四面体 (Lagrange要素として)
    # --- ↑↑↑ 追加ここまで ↑↑↑ ---
}

pv_mesh_manual = None # エラー発生時に備えて初期化

# mesh_from_meshio が正常にロードされている場合のみ実行 (セル5でロード済みのはず)
if 'mesh_from_meshio' in locals() and mesh_from_meshio is not None:
    try:
        points_for_manual = mesh_from_meshio.points
        pv_cells_list_manual = []
        pv_cell_types_list_manual = []
        converted_any_cells = False

        for cell_block in mesh_from_meshio.cells:
            meshio_cell_type_key = cell_block.type
            vtk_cell_type = meshio_str_to_pv_celltype.get(meshio_cell_type_key) # .get() を使うとキーがなくてもエラーにならない
            
            if vtk_cell_type is None:
                print(f"  手動変換スキップ: セルタイプ '{meshio_cell_type_key}' (VTKマッピング未定義)")
                continue
            
            print(f"  手動変換処理中: '{meshio_cell_type_key}' -> VTKタイプID {vtk_cell_type.value}")
            converted_any_cells = True
            num_points_per_cell = cell_block.data.shape[1]
            
            # 'tetra20' の場合、節点数が20であることを確認 (meshioが正しくパースしていれば一致するはず)
            if meshio_cell_type_key == "tetra20" and num_points_per_cell != 20:
                print(f"    警告: 'tetra20' と認識されましたが、節点数が {num_points_per_cell} です (期待値: 20)。処理は続行します。")

            for single_cell_node_indices in cell_block.data:
                pv_cells_list_manual.extend([num_points_per_cell, *single_cell_node_indices])
                pv_cell_types_list_manual.append(vtk_cell_type.value)

        if not converted_any_cells:
            print("  手動変換に適したセルが見つかりませんでした。")
        else:
            pv_mesh_manual = pv.UnstructuredGrid(
                np.array(pv_cells_list_manual, dtype=pv.ID_TYPE),
                np.array(pv_cell_types_list_manual, dtype=np.uint8),
                points_for_manual
            )
            print("\nPyVista メッシュオブジェクト情報 (手動変換):")
            print(pv_mesh_manual)

            plotter_manual = pv.Plotter(notebook=True, window_size=[800, 600])
            plotter_manual.add_mesh(pv_mesh_manual, show_edges=True, color="lightcoral")
            plotter_manual.view_isometric()
            plotter_manual.enable_zoom_style()
            plotter_manual.show_axes()
            plotter_manual.add_title(f"PyVista: {FILE_NAME} (手動変換)")
            
            print("\nJupyterセル内に手動変換プロットを表示します...")
            plotter_manual.show()

    except Exception as e:
        print(f"手動変換またはプロット中にエラー発生: {e}")
        traceback.print_exc()
else:
    print("`mesh_from_meshio` が存在しないため、ステップ3 (手動変換) をスキップします (ステップ1でエラーが発生した可能性があります)。")

In [ ]:
print("\n--- ステップ 3 (オプション): `meshio` オブジェクトから `PyVista UnstructuredGrid` への手動変換 ---")

# meshioのセルタイプ文字列をPyVistaのVTKセルタイプEnumにマッピング
meshio_str_to_pv_celltype = {
    "vertex": pv.CellType.VERTEX,
    "line": pv.CellType.LINE,
    "triangle": pv.CellType.TRIANGLE,
    "quad": pv.CellType.QUAD,
    "tetra": pv.CellType.TETRA,
    "pyramid": pv.CellType.PYRAMID,
    "wedge": pv.CellType.WEDGE,
    "hexahedron": pv.CellType.HEXAHEDRON,
    # --- ↓↓↓ この行が重要です！正しく記述されているか確認してください ↓↓↓ ---
    "tetra10": pv.CellType.QUADRATIC_TETRA, # 10節点二次四面体 (参考)
    "tetra20": pv.CellType.LAGRANGE_TETRAHEDRON, # 20節点四面体 (Lagrange要素として)
    # --- ↑↑↑ 確認ここまで ↑↑↑ ---
}

pv_mesh_manual = None # エラー発生時に備えて初期化

# mesh_from_meshio が正常にロードされている場合のみ実行
if mesh_from_meshio is not None:
    try:
        points_for_manual = mesh_from_meshio.points
        pv_cells_list_manual = []
        pv_cell_types_list_manual = []
        converted_any_cells = False

        for cell_block in mesh_from_meshio.cells:
            meshio_cell_type_key = cell_block.type
            vtk_cell_type = meshio_str_to_pv_celltype.get(meshio_cell_type_key)
            if vtk_cell_type is None:
                print(f"  手動変換スキップ: セルタイプ '{meshio_cell_type_key}' (VTKマッピング未定義)")
                continue
            
            print(f"  手動変換処理中: '{meshio_cell_type_key}' -> VTKタイプID {vtk_cell_type.value}")
            converted_any_cells = True
            num_points_per_cell = cell_block.data.shape[1]
            for single_cell_node_indices in cell_block.data:
                # PyVistaのセル配列形式: [点の数, 点0のインデックス, 点1のインデックス, ...]
                pv_cells_list_manual.extend([num_points_per_cell, *single_cell_node_indices])
                pv_cell_types_list_manual.append(vtk_cell_type.value) # VTKのセルタイプID (整数)

        if not converted_any_cells:
            print("  手動変換に適したセルが見つかりませんでした。")
        else:
            # PyVistaのUnstructuredGridオブジェクトを作成
            # cells配列は1Dのnumpy配列 (整数)、cell_types配列も1Dのnumpy配列 (uint8)
            pv_mesh_manual = pv.UnstructuredGrid(
                np.array(pv_cells_list_manual, dtype=pv.ID_TYPE), # セルの接続情報
                np.array(pv_cell_types_list_manual, dtype=np.uint8), # セルタイプ
                points_for_manual # 点の座標
            )
            print("\nPyVista メッシュオブジェクト情報 (手動変換):")
            print(pv_mesh_manual)

            # 手動変換したメッシュをPyVistaで可視化
            plotter_manual = pv.Plotter(notebook=True, window_size=[800, 600])
            plotter_manual.add_mesh(pv_mesh_manual, show_edges=True, color="lightcoral")
            plotter_manual.view_isometric()
            plotter_manual.enable_zoom_style()
            plotter_manual.show_axes()
            plotter_manual.add_title(f"PyVista: {FILE_NAME} (手動変換)")
            
            print("\nJupyterセル内に手動変換プロットを表示します...")
            plotter_manual.show()

    except Exception as e:
        print(f"手動変換またはプロット中にエラー発生: {e}")
        traceback.print_exc()
else:
    print("`mesh_from_meshio` がNoneのため、ステップ3をスキップします (ステップ1でエラーが発生した可能性があります)。")